In [1]:
import pandas as pd

In [2]:
#Load export data (The data come from another code i wrote)
df = pd.read_excel("FULL_DF_HS6.xlsx")

In [4]:
#Initialization of variables
unique_countries_count = df['TERRITORIO'].nunique()
print("Number of unique values in the 'Country' column:", unique_countries_count)
df = df.drop(columns="Unnamed: 0")
first_column = df.columns[2:-2].tolist()
countries = df.iloc[:130, 0]
dfprody = pd.DataFrame({"HS CODE": first_column})
dfexpy = pd.DataFrame({"Country": countries})

Number of unique values in the 'Country' column: 130


## Write the function to compute EXPY,PRODY and EXPY decomposition

In [8]:
def elementwise_expy(df, prody, year):
    total_sum_list = []
    df = df[df["year"] == year]

    #print(df.shape[0])
    for index, row in df.iterrows():
        # Extract data for the current row
        row_data = row.iloc[2:-2].tolist()  # Exclude the first column and last 3 elements
        #print(len(row_data))
        # Calculate the sum of the row
        total_sum_row = sum(row_data)
        #print(total_sum_row)
        
        # Normalize the row data
        normalized_row_data = [value / total_sum_row for value in row_data]
        
        # Multiply normalized row data with prody element-wise
        result_row = [x * y for x, y in zip(normalized_row_data, prody)]
        
        # Calculate the sum of the result row
        total_sum_result_row = sum(result_row)
        
        # Append the total sum of the result row to the list
        total_sum_list.append(total_sum_result_row)
        
    
  
    return total_sum_list

In [9]:
def elementwise_expy_scomposition2(df, prody, year, regione):
    total_sum_list = []
    results_interm = []
    df = df[(df["year"] == year) & (df["TERRITORIO"] == regione)]

    for index, row in df.iterrows():
        # Extract data for the current row
        row_data = row.iloc[2:-2].tolist()  # Exclude the first column and last 3 elements
        # Calculate the sum of the row
        total_sum_row = sum(row_data)
        # Normalize the row data
        normalized_row_data = [value / total_sum_row for value in row_data]
        # Multiply normalized row data with prody element-wise
        result_row = [x * y for x, y in zip(normalized_row_data, prody)]
        # Calculate the sum of the result row
        total_sum_result_row = sum(result_row)
        # Append the total sum of the result row to the list
        total_sum_list.append(total_sum_result_row)
        results_interm.append(result_row)

    total_sum_intermediate = sum(map(sum, results_interm))
    return total_sum_intermediate

In [10]:
def compute_prody(df, year):
    total_sum_per_column = {}
    results_per_column = {}
    printed_values = []

    df = df[df["year"] == year]

    # Iterate over each column from 1 to 65
    for i in range(2, (df.shape[1]-2)):
        column_name = df.columns[i]
        total_sum = 0
        results_list = []

        # Iterate over each row in the DataFrame
        for index, row in df.iterrows():
            product_total = row[column_name]
            export_value = row['Total Exports']

            if product_total != 0:  # Avoid division by zero
                result = product_total / export_value
                total_sum += result
                results_list.append(result)
            else:
                results_list.append(0)

        # Check if total_sum is zero to avoid division by zero
        if total_sum != 0:
            # Divide each element of the list by the total sum and multiply by the 'GDP' column
            normalized_and_scaled_results = []
            for j, result in enumerate(results_list):
                # Get the GDP value corresponding to the current row
                GDP_value = df.loc[df.index[j], 'GDP']
                # Calculate the normalized and scaled result
                normalized_result = result / total_sum * GDP_value
                normalized_and_scaled_results.append(normalized_result)

            sum_normalized_and_scaled_results = sum(normalized_and_scaled_results)
        else:
            # If total_sum is zero, all normalized results are zero
            normalized_and_scaled_results = [0] * len(results_list)
            sum_normalized_and_scaled_results = 0

        total_sum_per_column[column_name] = total_sum
        results_per_column[column_name] = normalized_and_scaled_results
        printed_values.append(sum_normalized_and_scaled_results)

    return printed_values

## Compute PRODY and EXPY for every years

In [13]:
years = [2017, 2018, 2019, 2020, 2021, 2022]

# Loop through each year
for year in years:
    # Compute prody and expy for the current year
    prody = compute_prody(df, year)
    expy = elementwise_expy(df, prody, year)
    
    # Convert the year to a string for column naming
    year_str = str(year)
    
    # Add expy as a new column to dfexpy with the specified year
    dfexpy = dfexpy.assign(**{year_str: expy})
    
    # Add prody as a new column to dfprody with the specified year
    dfprody = dfprody.assign(**{year_str: prody})

# 2023 has to be handled seperately due to missing value in exports for some countries

In [15]:
year= "2023"
prody = compute_prody(df, 2023)
expy = elementwise_expy(df, prody, 2023)


In [16]:
df23 = df[df["year"]== 2023]
first_column = df23.iloc[:, 0]
dfexpy23 = pd.DataFrame({"Country": first_column})
year = "2023"
dfprody = dfprody.assign(**{year: prody})
dfexpy23 = dfexpy23.assign(**{year: expy})

In [17]:
#Merge all years together
dfexpy = pd.merge(dfexpy, dfexpy23, on='Country', how='outer')
dfexpy

,Country,2017,2018,2019,2020,2021,2022,2023
0,Albania,30862.626241,39711.218098,31537.429882,32288.808743,33673.258577,51881.033534,NaN
1,Andorra,39511.316159,42520.251007,44207.597039,42005.566280,44287.239863,50663.561131,56474.657509
2,Angola,26817.729230,27489.263815,28168.920180,25507.653022,27646.225612,31594.810927,NaN
3,Antigua and Barbuda,25997.951625,26826.924398,26218.772036,24216.094619,29212.429572,32453.896755,36256.532388
4,Argentina,24005.067685,29367.042965,26746.663500,26694.684346,30623.972575,32138.714616,33142.287996
...,...,...,...,...,...,...,...,...
125,Uruguay,23464.399050,25224.398939,25586.939887,24812.261723,27374.898986,30072.427662,33127.200015
126,Uzbekistan,17939.901685,19928.310888,20090.679004,16344.785943,19588.881893,22644.518833,NaN
127,Viet Nam,25904.747253,27845.691976,32473.675739,29575.074561,33478.491068,33746.308030,NaN
128,Zambia,11629.837775,12502.311171,11726.457770,11546.664434,12677.698653,14375.818023,15840.006662


# Add code description to each HScode

In [18]:
names = pd.read_excel('Source/HSCodeandDescription.xlsx', sheet_name=1)
names = names[names["Level"]==6]
names = names[['Code', 'Description']]
names.rename(columns={'Code': 'CTCI'}, inplace=True)
names['CTCI'] = names['CTCI'].str.zfill(6)
names.rename(columns={'CTCI': 'HS CODE'}, inplace=True)

In [19]:
# If 'CTCI' is not already string type, convert it to string
dfprody['HS CODE'] = dfprody['HS CODE'].astype(str)

# Add leading zeros to make each value 6 characters long
dfprody['HS CODE'] = dfprody['HS CODE'].str.zfill(6)

In [20]:
dfprody = pd.merge(dfprody, names, how='left', on='HS CODE')

In [33]:
#Export the prody evolution
#dfprody.to_excel("PRODY_EVOLUTION_HS6_WORLD_NEW.xlsx")

# SCOMPOSITION

In [22]:
prodydf = pd.read_excel("PRODY_EVOLUTION_HS6_WORLD_NEW.xlsx")
prodydf = prodydf.drop(columns="Unnamed: 0")

In [23]:
countries = df.iloc[:130, 0]

In [24]:
#Scompose EXPY in its component to better understand the movement
def elementwise_expy_scomposition(df, prody, year, regione):
    total_sum_list = []
    results_interm = []
    df = df[df["TERRITORIO"] == regione]
    df = df[df["year"] == year]
    print(df.shape)
    for index, row in df.iterrows():
        # Extract data for the current row
        row_data = row.iloc[2:-2].tolist()  # Exclude the first column and last 3 elements
        
        # Calculate the sum of the row
        total_sum_row = sum(row_data)
        
        # Normalize the row data
        normalized_row_data = [value / total_sum_row for value in row_data]
        
        # Multiply normalized row data with prody element-wise
        result_row = [x * y for x, y in zip(normalized_row_data, prody)]
        
        # Calculate the sum of the result row
        total_sum_result_row = sum(result_row)
        
        # Append the total sum of the result row to the list
        total_sum_list.append(total_sum_result_row)
        results_interm.append(result_row)
    
    return results_interm



CTCI = []
for i in range(2, (df.shape[1]-2)):
    column_name = df.columns[i]
    CTCI.append(column_name)


#TRASFORM A LIST OF LIST INTO A LIST
def flatten_extend(matrix):
     flat_list = []
     for row in matrix:
         flat_list.extend(row)
     return flat_list

In [25]:
prodyy = prodydf["2017"].to_list()

In [26]:
#TESTING IF THE VALUE MATCHES
year = "2017"
prody = prodydf["2017"].to_list()
regione = "Italy"
expy = elementwise_expy_scomposition(df, prody, 2017, regione)
expy
flat_expy = [item for sublist in expy for item in sublist]

# Calculate the sum of the flattened list
list_sum = sum(flat_expy)

# Print the sum
print("Sum of the list:", list_sum)

(1, 5399)
Sum of the list: 34211.08180462537


In [27]:
column_position = 5396

# Check if the position is within the range of column indices
if column_position < len(df.columns):
    column_name = df.columns[column_position]
    print(f"The column name at position {column_position} is: {column_name}")
else:
    print(f"Position {column_position} is out of range. The DataFrame has {len(df.columns)} columns.")

The column name at position 5396 is: 40811


## Compute decomposition for each year

In [32]:
import numpy as np

#Compute the percentage of expy and prody for each country
def iterate_list(my_list, df):
    dfs = []  # List to store DataFrames

    # List of years to process
    years = [2017, 2018, 2019, 2020, 2021, 2022, 2023]

    for element in my_list:
        dfexpysc = pd.DataFrame({"CTCI": CTCI})
        dfexpysc["Country"] = element
        
        for year in years:
            year_str = str(year)

            # Compute prody and expy for the current year and region
            prody = prodydf[year_str].to_list()
            expy = elementwise_expy_scomposition(df, prody, year, element)
            expy = flatten_extend(expy)
            
            # Handle case where expy is empty for 2023
            if year == 2023 and not expy:
                expy = [np.nan] * 5395
            
            # Add expy as a new column
            dfexpysc = dfexpysc.assign(**{f"{year_str} expy": expy})

            # Get exportations for the current year and region
            esp = df[(df["TERRITORIO"] == element) & (df["year"] == year)]
            if year == 2023 and esp.empty:
                esp = [np.nan] * 5395
            else:
                esp = esp.values[0, 2:5397]
            
            # Add esp as a new column
            dfexpysc = dfexpysc.assign(**{f"{year_str} esp": esp})

            # Compute and add percentage expy
            tot_expy = dfexpysc[f"{year_str} expy"].sum()
            dfexpysc[f"{year_str} percentage EXPY"] = (dfexpysc[f"{year_str} expy"] / tot_expy) if tot_expy != 0 else np.nan

            # Compute and add percentage esp
            tot_esp = dfexpysc[f"{year_str} esp"].sum()
            dfexpysc[f"{year_str} percentage esp"] = (dfexpysc[f"{year_str} esp"] / tot_esp) if tot_esp != 0 else np.nan

        # Append the DataFrame to the list
        dfs.append(dfexpysc)

    return dfs

In [33]:
#RUN THE FUNCTION
countries = df.iloc[:130, 0]
A = iterate_list(countries, df)

(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(0, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(0, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(0, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(0, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(0, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(0, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(1, 5399)
(0, 5399)
(1, 5399)
(1, 5399)


In [34]:
#ADD THE CODE DESCRIPTION
names = pd.read_excel('Source/HSCodeandDescription.xlsx', sheet_name=1)
names = names[names["Level"]==6]
names = names[['Code', 'Description']]
names.rename(columns={'Code': 'CTCI'}, inplace=True)
names['CTCI'] = names['CTCI'].str.zfill(6)
#names.rename(columns={'CTCI': 'HS CODE'}, inplace=True)

In [35]:
# Iterate through each dataframe in the list
for i in range(len(A)):
    # If 'CTCI' is not already string type, convert it to string
    A[i]['CTCI'] = A[i]['CTCI'].astype(str)
    
    # Add leading zeros to make each value 6 characters long
    A[i]['CTCI'] = A[i]['CTCI'].str.zfill(6)
    
    # Merge the dataframe with 'names' on the 'CTCI' column
    A[i] = pd.merge(A[i], names, how='left', on='CTCI')

In [36]:
#EXPORT TO EXCEL
def export_to_excel(list_of_dfs, file_name):
    with pd.ExcelWriter(file_name) as writer:
        for i, df in enumerate(list_of_dfs):
            sheet_name = list_of_dfs[i]["Country"].iloc[0]  # Extract the element from the DataFrame
            df.to_excel(writer, sheet_name=sheet_name, index=False)

# Example usage:
#export_to_excel(A, "Prova.xlsx")

In [37]:
export_to_excel(A, "FULL_HS6_EXPY_SCOMPOSED_NEW.xlsx")